In [1]:
import pandas as pd

In [4]:
direct_labeling_fullnames = set(pd.read_csv('../data/interim/direct_labeling_strategy_4s.csv', index_col=0).fullname)

In [5]:
len(direct_labeling_fullnames)

4587

In [6]:
with pd.read_json('../data/interim/labeling_contributions_preprocessed_no_bot.jsonl', orient='records', lines=True, chunksize=500) as reader:
    df = pd.concat(chunk[chunk.fullname.isin(direct_labeling_fullnames)][[ 'author', 'created_utc',
           'subreddit','text', 'fullname', 'parent_fullname',
           'link_fullname',]] for chunk in reader)

In [8]:
parent_fullnames = set(df.parent_fullname)
parents = list()
for chunk in pd.read_json('../data/interim/labeling_subthread_all_filtered_preprocessed_no_bot.jsonl.zip', compression='zip', chunksize=1000, lines=True):
    parents.append(chunk[chunk.fullname.isin(parent_fullnames)])
parent_df = pd.concat(parents)
len(parent_df)

4573

In [9]:
parent_df.author.nunique()

3874

Tunderbar1
moderator of skullduggery_theorems: Examining conspiracy theories in an free and open society.
moderator of canada_unfiltered: Open discussion of politics in Canada

ee4m's most recent posts are in JordanPeterson, on threads like
ee4m commented on The WEF is HIDING bugs in these foods and you don't even know it | Redacted with Clayton Morris

aceofbase_in_ur_mind has been suspended

In [12]:
parent_df.groupby('author').size().sort_values(ascending=False)

author
[deleted]               612
Tunderbar1                5
ee4m                      4
aceofbase_in_ur_mind      4
makingacanadian           3
                       ... 
Neeentendo64              1
NeiliusAntitribu          1
NeoFury84                 1
NeoShweaty                1
zsreport                  1
Length: 3874, dtype: int64

In [13]:
parent_authors = set(parent_df.author.unique())

In [15]:
parent_authors.discard('[deleted]')

In [16]:
parent_authors.discard('[removed]')

In [19]:
with open('../data/raw/botnames_expanded.txt') as f:
    botnames =  set(i.strip() for i in f)

In [20]:
parent_authors = list(sorted(i for i in parent_authors if i not in botnames))

In [21]:
len(parent_authors)

3869

In [22]:
with open('../data/interim/labeled_authors.csv', 'w+') as f:
    f.write('/n'.join(parent_authors))

In [23]:
parent_df

,archived,author,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_text_color,author_flair_type,brand_safe,can_gild,...,subcaption,quarantined,associated_award,collapsed_because_crowd_control,comment_type,collapsed_reason_code,author_is_blocked,editable,unrepliable_reason,ad_promoted_user_posts
629,True,[deleted],,None,NaN,None,dark,NaN,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2542,True,unpatriotic,None,None,[],None,None,text,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4381,NaN,kidt82,NaN,None,NaN,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5877,NaN,Dizzy_Slip,NaN,None,NaN,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6455,1.0,Slipgrid,None,None,[],None,None,text,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9056314,False,[deleted],,None,NaN,None,dark,NaN,NaN,True,...,NaN,NaN,NaN,NaN,NaN,DELETED,NaN,NaN,NaN,NaN
9056999,False,Limonov-nyan,None,None,[],None,None,text,NaN,True,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
9057154,False,stockyardtrash,None,None,[],None,None,text,NaN,True,...,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN
9057325,False,[deleted],,None,NaN,None,dark,NaN,NaN,True,...,NaN,NaN,NaN,NaN,NaN,DELETED,NaN,NaN,NaN,NaN


In [30]:
parent_df[~parent_df.author.isin(botnames|{'[removed]', '[deleted]'})].to_json('../data/interim/labeled_contributions_nobots.jsonl', lines=True, orient='records')

In [31]:
parent_df.to_json('../data/interim/labeled_contributions.jsonl', lines=True, orient='records')